In [1]:
import geopandas as gp
from shapely.geometry import LineString, Point
import rtree
import os
import numpy as np

In [2]:
def build_load_idx(seg_idx_path, segs_with_ids, update=False):
    """
    params:
        seg_idx_path: the path of segment index if the index doesn't exist, the index will be built.
        segs_with_ids: dict: key=objid, value=shapely.geometry.LineString, segments that will be indexed
        update: True: rebuild the index no matter whether the index exists
    Output:
        rtree index of given segments.
    """
    idx_exist = os.path.isfile(seg_idx_path+'.dat') & os.path.isfile(seg_idx_path+'.idx') 
    if update and idx_exist:
        print('remove existing files')
        os.remove(seg_idx_path+'.dat')
        os.remove(seg_idx_path+'.idx')
        idx_exist = False
    seg_idx = rtree.index.Rtree(seg_idx_path)
    if not idx_exist:
        print('building idx')
        for objid,seg in segs_with_ids.items():
            seg_idx.insert(objid, seg.bounds)
        print('built idx')

    return seg_idx

def pt2seg(pt, segs_with_ids, seg_idx):
    """
    params: 
        pt: shapely.geometry.Point
        segs_with_ids:
        seg_idx: rtree index of segments
    Process:
        1. load segment index and find segments(var:idx_match) whose bounds intersect with point's bounds
           if there isn't any matched segment, the point will be buffered until len of idx_match !=0
        2. find the segments that actually intersect with given point
           if there isn't any intersected segment, nearest segment will be returned
    Output:
       hint: whether the returned segid is "nearest" or "intersected"
       segid: assigned segment id
       cnt: len of candidates 
       candidates: the index of segments that are "equally near"/"intersected" to the point
       distance: the distance between point and segment
    """
    
    idx_match = list(seg_idx.intersection(pt.bounds))

    # in case no idx is matched
    bfr_value = 0.000001
    while len(idx_match)==0:
        idx_match = list(seg_idx.intersection(pt.buffer(bfr_value).bounds))
        bfr_value*=2
    
    fil = [pt.intersects(segs_with_ids[idx]) for idx in idx_match]
    seg_match = [idx for (idx,b) in zip(idx_match, fil) if b]
    
    if len(seg_match)==0:
        dis = np.array([pt.distance(segs_with_ids[idx]) for idx in idx_match])
        minimum = dis.min()
        segid = idx_match[dis.argmin()]
        return 'nearest', segid, int(np.where(dis==minimum)[0].shape[0]), np.array(idx_match)[np.where(dis==minimum)[0]],minimum
    else:
        segid=seg_match[0]
        return 'intersected', segid, len(seg_match), np.array(seg_match),0

In [16]:
dc_segments = gp.read_file("../dc-maps-master/maps/street-segments.geojson")
vision0_pts = gp.read_file("../dc-maps-master/maps/vision-zero-safety-transportation.geojson")
seg_with_id = dict(zip(dc_segments.STREETSEGID, dc_segments.geometry))
dc_seg_idx_path = "dc_segment_index"
dc_seg_idx = build_load_idx(dc_seg_idx_path, seg_with_id)

NameError: name 'build_load_idx' is not defined

In [7]:
from itertools import chain

In [4]:
dc_segments = gp.read_file("../dc-maps-master/maps/street-segments.geojson")
vision0_pts = gp.read_file("../dc-maps-master/maps/vision-zero-safety-transportation.geojson")

In [27]:
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    from math import radians, cos, sin, asin, sqrt
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    km = 6367 * c
    m = km *1000
    return m   
def ptfromln(pt, ln):
    n_pt = ln.interpolate(ln.project(pt))
    lon1, lat1 = n_pt.coords[0]
    lon2, lat2 = pt.coords[0]
    return haversine(lon1, lat1, lon2, lat2)
def pts2seg(pts, gp_segs, buffer_dis=50, near_dis_thres=5):
    pts_crs,gp_segs_crs = pts.to_crs(epsg=3559), gp_segs.to_crs(epsg=3559)
    pts_crs_bfr = pts_crs.copy()

    pts_crs_bfr.geometry = pts_crs_bfr.buffer(near_dis_thres*1.1)
    close_jn = gp.tools.sjoin(pts_crs_bfr, gp_segs_crs)[['OBJECTID_left','STREETSEGID_right']]
    handledid = set(pd.unique(close_jn.OBJECTID_left))
    mask = (~pts_crs_bfr.OBJECTID.isin(handledid))

    far_jns = []
    while pts_crs_bfr[mask].shape[0]!=0:
        pts_crs_bfr.loc[mask, 'geometry'] = pts_crs_bfr[mask].buffer(buffer_dis)
        jn = gp.tools.sjoin(pts_crs_bfr[mask], gp_segs_crs)[['OBJECTID_left','STREETSEGID_right']]
        far_jns.append(jn)
        handledid |= set(pd.unique(jn.OBJECTID_left))
        mask = (~pts_crs_bfr.OBJECTID.isin(handledid))
        
    far_jns = pd.concat(far_jns)
    mr = pd.merge(gp_segs[['geometry','STREETSEGID']],far_jns , left_on='STREETSEGID', right_on='STREETSEGID_right')
    mr = pd.merge(pts[['OBJECTID','geometry','STREETSEGID']],mr, left_on='OBJECTID', right_on='OBJECTID_left')
    mr['dis']=mr.apply(lambda x: ptfromln(x.geometry_x, x.geometry_y),axis=1)
    result = close_jn.groupby('OBJECTID_left')['STREETSEGID_right'].apply(list).append(mr.groupby('OBJECTID').apply(lambda x: [x.ix[x.dis.idxmin()].STREETSEGID_y]))
    return pd.DataFrame(result, columns=['segid'])

In [104]:
dc_segs_statistic = dc_segments[['OBJECTID','STREETSEGID','SHAPE_Length','geometry']]

In [105]:
dc_segs_statistic = dc_segs_statistic.merge(vision0_pts, left_on = 'STREETSEGID', right_index=True, how='left')

In [14]:
dc_segs_statistic = dc_segs_statistic.dropna()

In [15]:
dc_segs_statistic.head().T

,0,4,16,19,25
OBJECTID_x,1,5,17,20,26
STREETSEGID_x,876,2216,2902,4213,179
SHAPE_Length,192.925,119.19,126.698,119.102,153.46
geometry_x,LINESTRING (-77.01239193659748 38.950836102255...,LINESTRING (-77.01161670521323 38.956595348877...,LINESTRING (-77.01324339541158 38.951215745942...,LINESTRING (-77.01357027112063 38.954497383391...,LINESTRING (-77.01368810906661 38.961335901924...
COMMENTS,Cyclists constantly use the sidewalk up and do...,Traffic going up this hill is generally too fa...,This intersection has a fair number of pedestr...,Vehicles picking up and dropping off passenger...,I was hit by a car here when I was in the cycl...
GLOBALID,{17DD828B-8D39-4B7A-88EB-2125E60DA1EA},{BD16F1DE-92F0-4B6B-ACEA-BA904EF18683},{15AC14A9-746F-4F75-B2FA-C173260C5855},{0F7D5B3F-267F-4D2C-9D53-71D06B2F6D5C},{20B961FB-7304-44AE-BCC5-775E17675DA6}
OBJECTID_y,955,3299,1974,5688,122
REQUESTDATE,2015-07-02T12:58:50,2015-07-17T19:58:09,2015-07-10T02:30:00,2015-08-15T22:42:59,2015-06-29T20:53:04
REQUESTID,955,3299,1974,5688,122
REQUESTTYPE,Cyclist behavior,Speeding,Failure to stop for pedestrians,Double parking,Other Driving Issue


In [17]:
dc_segs_statistic[dc_segs_statistic.STREETSEGID_x == 2216]

,OBJECTID_x,STREETSEGID_x,SHAPE_Length,geometry_x,COMMENTS,GLOBALID,OBJECTID_y,REQUESTDATE,REQUESTID,REQUESTTYPE,STATUS,STREETSEGID_y,USERTYPE,geometry_y
4,5,2216,119.190312,LINESTRING (-77.01161670521323 38.956595348877...,Traffic going up this hill is generally too fa...,{BD16F1DE-92F0-4B6B-ACEA-BA904EF18683},3299.0,2015-07-17T19:58:09,3299,Speeding,Unassigned,5942.0,Biker,POINT (-77.03185508121238 38.9208823767754)


In [18]:
dc_segs_statistic

,OBJECTID_x,STREETSEGID_x,SHAPE_Length,geometry_x,COMMENTS,GLOBALID,OBJECTID_y,REQUESTDATE,REQUESTID,REQUESTTYPE,STATUS,STREETSEGID_y,USERTYPE,geometry_y
0,1,876,192.925131,LINESTRING (-77.01239193659748 38.950836102255...,Cyclists constantly use the sidewalk up and do...,{17DD828B-8D39-4B7A-88EB-2125E60DA1EA},955.0,2015-07-02T12:58:50,955,Cyclist behavior,Unassigned,11399.0,Pedestrian,POINT (-77.03195395364403 38.90678948383844)
4,5,2216,119.190312,LINESTRING (-77.01161670521323 38.956595348877...,Traffic going up this hill is generally too fa...,{BD16F1DE-92F0-4B6B-ACEA-BA904EF18683},3299.0,2015-07-17T19:58:09,3299,Speeding,Unassigned,5942.0,Biker,POINT (-77.03185508121238 38.9208823767754)
16,17,2902,126.698028,LINESTRING (-77.01324339541158 38.951215745942...,This intersection has a fair number of pedestr...,{15AC14A9-746F-4F75-B2FA-C173260C5855},1974.0,2015-07-10T02:30:00,1974,Failure to stop for pedestrians,Unassigned,7465.0,Pedestrian,POINT (-77.02606355519816 38.9686399653321)
19,20,4213,119.102319,LINESTRING (-77.01357027112063 38.954497383391...,Vehicles picking up and dropping off passenger...,{0F7D5B3F-267F-4D2C-9D53-71D06B2F6D5C},5688.0,2015-08-15T22:42:59,5688,Double parking,Unassigned,233.0,Car Driver,POINT (-77.00434504379416 38.88604499110777)
25,26,179,153.460480,LINESTRING (-77.01368810906661 38.961335901924...,I was hit by a car here when I was in the cycl...,{20B961FB-7304-44AE-BCC5-775E17675DA6},122.0,2015-06-29T20:53:04,122,Other Driving Issue,Unassigned,1067.0,Biker,POINT (-77.03240650725611 38.89551047123661)
35,36,277,205.857560,LINESTRING (-77.01368063507392 38.900211463306...,An unsignalized pedestrian crossing should be ...,{2A1C21F0-8AFD-4168-9E7F-4CB6CEFB709C},281.0,2015-06-30T14:43:34,281,Other Walking Issue,Unassigned,496.0,Pedestrian,POINT (-77.03024304187144 38.90858849023896)
39,40,1440,64.684222,LINESTRING (-77.01427336215276 38.914996139209...,As soon as this section of Potomac because 2-3...,{09BEF8F1-9B67-416F-8F89-EBC8557597AC},2818.0,2015-07-16T12:33:09,2818,Speeding,Unassigned,1114.0,Pedestrian,POINT (-76.97900052357966 38.88302359375337)
42,43,1750,79.779456,LINESTRING (-77.0144223920466 38.9162872196654...,Trucks/delivery vehicles routinely park in the...,{735F6A22-06DC-41AB-B062-12A10CC6CEAA},3062.0,2015-07-16T20:19:44,3062,Double parking,Unassigned,7775.0,Biker,POINT (-77.04293670047392 38.90725022523262)
45,46,667,61.926531,LINESTRING (-77.01462766975995 38.918114804591...,Red lights along this stretch of 4th Street. a...,{48682AFE-D20C-40B4-8B4C-A775DBE5E393},642.0,2015-07-01T17:10:13,642,Other Biking Issue,Unassigned,6574.0,Biker,POINT (-77.01755746826102 38.88606030948256)
47,48,2308,54.452949,LINESTRING (-77.01475138888762 38.919182095956...,There are numerous potholes on this street whi...,{088DF38A-9886-4987-8D71-EE394131BD3F},3971.0,2015-07-28T13:13:04,3971,Other Driving Issue,Unassigned,11085.0,Pedestrian,POINT (-76.9536437737025 38.88724114859725)


In [20]:
dc_segs_statistic[dc_segs_statistic.STREETSEGID_x == 644]

,OBJECTID_x,STREETSEGID_x,SHAPE_Length,geometry_x,COMMENTS,GLOBALID,OBJECTID_y,REQUESTDATE,REQUESTID,REQUESTTYPE,STATUS,STREETSEGID_y,USERTYPE,geometry_y
4435,4436,644,275.843424,LINESTRING (-77.03848871583934 38.911133414859...,South-side sidewalk from 6th to 9th is impassi...,{37A25004-7921-4DB9-B02C-B7CACEED222F},609.0,2015-07-01T16:56:46,609,Accessibility Issue,Unassigned,13176.0,Pedestrian,POINT (-76.99616576427324 38.90524480982609)


In [106]:
dc_segs_statistic = dc_segments[['OBJECTID','STREETSEGID','SHAPE_Length','geometry']]
dc_segs_statistic = dc_segs_statistic.merge(vision0_pts, left_on = 'STREETSEGID', right_index=True, how='left')

In [107]:
vision0_pts_seg = pts2seg(vision0_pts, dc_segments)

In [28]:
import pandas as pd

In [108]:
seg_counts = pd.DataFrame(pd.Series(list(chain.from_iterable(vision0_pts_seg.values.ravel()))).value_counts(),columns=["vision0_pts_seg"])

In [109]:
dc_segs_statistic = dc_segments[['OBJECTID','STREETSEGID','SHAPE_Length','geometry']]
dc_segs_statistic = dc_segs_statistic.merge(seg_counts, left_on = 'STREETSEGID', right_index=True, how='left')

In [33]:
dc_segs_statistic.head().T

,0,1,2,3,4
OBJECTID,1,2,3,4,5
STREETSEGID,876,13161,8240,10395,2216
SHAPE_Length,192.925,131.936,117.887,108.331,119.19
geometry,LINESTRING (-77.01239193659748 38.950836102255...,LINESTRING (-77.01163764014346 38.953383268855...,LINESTRING (-77.01157276320811 38.954558133776...,LINESTRING (-77.0116155077942 38.9556194835925...,LINESTRING (-77.01161670521323 38.956595348877...
vision0_pts_seg,2,1,1,NaN,NaN


In [34]:
dc_segs_statistic.dropna().head().T

,0,1,2,10,14
OBJECTID,1,2,3,11,15
STREETSEGID,876,13161,8240,9116,12034
SHAPE_Length,192.925,131.936,117.887,124.981,124.618
geometry,LINESTRING (-77.01239193659748 38.950836102255...,LINESTRING (-77.01163764014346 38.953383268855...,LINESTRING (-77.01157276320811 38.954558133776...,LINESTRING (-77.01033664424322 38.904550094214...,LINESTRING (-77.01111526940367 38.904550711856...
vision0_pts_seg,2,1,1,1,1


In [36]:
dc_segs_statistic[dc_segs_statistic.STREETSEGID == 644]

,OBJECTID,STREETSEGID,SHAPE_Length,geometry,vision0_pts_seg
4435,4436,644,275.843424,LINESTRING (-77.03848871583934 38.911133414859...,3.0


In [110]:
dc_segs_statistic_drop = dc_segs_statistic.dropna()

In [38]:
dc_segs_statistic.shape

(13522, 5)

In [39]:
dc_segs_statistic_drop.shape

(4685, 5)

In [111]:
dc_segs_statistic_drop['vision0_pts/len'] = dc_segs_statistic_drop.vision0_pts_seg/dc_segs_statistic_drop.SHAPE_Length

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [92]:
def normalize(col_name, df):
    col_name_norm = col_name + '_norm'
    df[col_name_norm] = df[col_name]+1
    df[col_name_norm] = np.log2(df[col_name_norm])
    df[col_name_norm] = df[col_name_norm]/df[col_name_norm].max()

In [42]:
dc_segs_statistic_drop.head().T

,0,1,2,10,14
OBJECTID,1,2,3,11,15
STREETSEGID,876,13161,8240,9116,12034
SHAPE_Length,192.925,131.936,117.887,124.981,124.618
geometry,LINESTRING (-77.01239193659748 38.950836102255...,LINESTRING (-77.01163764014346 38.953383268855...,LINESTRING (-77.01157276320811 38.954558133776...,LINESTRING (-77.01033664424322 38.904550094214...,LINESTRING (-77.01111526940367 38.904550711856...
vision0_pts_seg,2,1,1,1,1
vision0_pts/len,0.0103667,0.00757945,0.0084827,0.00800124,0.00802451


In [112]:
normalize('vision0_pts_seg', dc_segs_statistic_drop)
normalize('vision0_pts/len', dc_segs_statistic_drop)

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexin

In [44]:
dc_segs_statistic_drop.head().T

,0,1,2,10,14
OBJECTID,1,2,3,11,15
STREETSEGID,876,13161,8240,9116,12034
SHAPE_Length,192.925,131.936,117.887,124.981,124.618
geometry,LINESTRING (-77.01239193659748 38.950836102255...,LINESTRING (-77.01163764014346 38.953383268855...,LINESTRING (-77.01157276320811 38.954558133776...,LINESTRING (-77.01033664424322 38.904550094214...,LINESTRING (-77.01111526940367 38.904550711856...
vision0_pts_seg,2,1,1,1,1
vision0_pts/len,0.0103667,0.00757945,0.0084827,0.00800124,0.00802451
vision0_pts_seg_norm,0.341303,0.215338,0.215338,0.215338,0.215338
vision0_pts/len_norm,0.0169106,0.012381,0.0138503,0.0130673,0.0131051


In [47]:
dc_segs_statistic_drop.sort('vision0_pts/len')

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,OBJECTID,STREETSEGID,SHAPE_Length,geometry,vision0_pts_seg,vision0_pts/len,vision0_pts_seg_norm,vision0_pts/len_norm
13371,13372,14688,6438.432414,LINESTRING (-77.01777214063064 38.818858999576...,1.0,0.000155,0.215338,0.000255
13434,13435,14747,3515.130385,LINESTRING (-77.01798919407902 38.882347633133...,1.0,0.000284,0.215338,0.000466
13370,13371,14671,2976.797197,LINESTRING (-76.95866864498795 38.889793996857...,1.0,0.000336,0.215338,0.000551
12701,12702,3363,4618.773679,LINESTRING (-77.02953799963242 38.878728644748...,2.0,0.000433,0.341303,0.000710
13387,13388,14707,3959.059073,LINESTRING (-76.95815731743873 38.889794931856...,2.0,0.000505,0.341303,0.000828
12015,12016,9529,1829.412094,LINESTRING (-77.00904606681496 38.926674486785...,1.0,0.000547,0.215338,0.000896
11908,11909,5024,1749.199090,LINESTRING (-77.03464270102016 38.879184367362...,1.0,0.000572,0.215338,0.000937
9258,9259,3727,3287.000458,LINESTRING (-76.95815731743873 38.889794931856...,2.0,0.000608,0.341303,0.000997
8467,8468,292,1269.559183,LINESTRING (-76.97244165561152 38.917336416995...,1.0,0.000788,0.215338,0.001291
11627,11628,8083,1232.016795,LINESTRING (-77.00326332854058 38.867912708876...,1.0,0.000812,0.215338,0.001330


In [113]:
def gradient_color(percent):
    min_color = np.array([255,255,255])
    max_color = np.array([248,105,107])
    return '#%02x%02x%02x' % tuple([int(k) for k in min_color+(max_color-min_color)*percent])

def write_var(col_name, var_name, df,f):
    df['color']=df[col_name].apply(gradient_color)
    f.write('var %s = %s;\n' % (var_name, df.to_json()))

with open('../visualization/Volumes of Vision0_test.js','w+') as f:
    write_var('vision0_pts_seg_norm', 'vision0_ptseg', dc_segs_statistic_drop,f)
    write_var('vision0_pts/len_norm', 'vision0_ptseg_norm', dc_segs_statistic_drop,f)

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [49]:
import geopandas as gp
from shapely.geometry import LineString, Point
import rtree
import os
import numpy as np
from itertools import chain

In [52]:
from pandas import DataFrame
from geopandas import GeoDataFrame
from shapely.geometry import Point

In [54]:
data = DataFrame.from_csv("../Cityworks_Service_Requests_filtered.csv", index_col=False)
points = [Point(row['X'], row['Y']) for key, row in data.iterrows()]
geo_df = GeoDataFrame(data,geometry=points)
fname = "../Cityworks_Service_Requests_filtered.geojson"
geo_df.to_file(fname, driver='GeoJSON')

//anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2825: DtypeWarning: Columns (15,23,26,32,40) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [7]:
city_req_pts = gp.read_file("../Cityworks_Service_Requests_filtered.geojson")

In [11]:
dc_seg_buffer_gp = dc_segments[['OBJECTID','STREETSEGID','geometry']].copy()
dc_seg_buffer_gp.geometry = dc_seg_buffer_gp.buffer(0.0001)
city_req_buffer_gp = city_req_pts[['OBJECTID','LONGITUDE','LATITUDE','geometry','INITIATEDDATE','ADDRESS']].copy()
city_req_buffer_gp.geometry = city_req_buffer_gp.buffer(0.0001)
joined = sjoin(city_req_buffer_gp, dc_seg_buffer_gp, how='left', op='intersects')
#gb = joined.reset_index().groupby('OBJECTID_LEFT').count()
    
#print(gb[gb.OBJECTID_right==0].shape[0], ' points spatially match no segment')
#print(gb[gb.OBJECTID_right==1].shape[0], ' points spatially match 1 segment')
#print(gb[gb.OBJECTID_right>1].shape[0], ' points spatially match multiple segments')

In [1]:
import geopandas as gp
from shapely.geometry import LineString, Point
import rtree
import os
import numpy as np

In [2]:
from geopandas.tools import sjoin
import pandas as pd
from IPython.display import display
import snap

In [8]:
from itertools import chain

In [12]:
joined.head().T

,0,1,1,1,1
OBJECTID_left,1,2,2,2,2
LONGITUDE,-77.0806,-77.0199,-77.0199,-77.0199,-77.0199
LATITUDE,38.9075,38.9661,38.9661,38.9661,38.9661
geometry,"POLYGON ((-77.08052689 38.90750167, -77.080527...","POLYGON ((-77.01980087999999 38.96614244, -77....","POLYGON ((-77.01980087999999 38.96614244, -77....","POLYGON ((-77.01980087999999 38.96614244, -77....","POLYGON ((-77.01980087999999 38.96614244, -77...."
INITIATEDDATE,2013-04-09T15:29:30,2014-12-13T16:13:19,2014-12-13T16:13:19,2014-12-13T16:13:19,2014-12-13T16:13:19
ADDRESS,1773 WILLARD STREET NW,SHERIDAN ST & 5TH STREET NW,SHERIDAN ST & 5TH STREET NW,SHERIDAN ST & 5TH STREET NW,SHERIDAN ST & 5TH STREET NW
index_right,1742,3991,318,319,3990
OBJECTID_right,1743,3992,319,320,3991
STREETSEGID,7076,3338,2333,143,6957


In [13]:
gb = joined.reset_index().groupby('OBJECTID_left').count()

In [14]:
print(gb[gb.OBJECTID_right==0].shape[0], ' points spatially match no segment')
print(gb[gb.OBJECTID_right==1].shape[0], ' points spatially match 1 segment')
print(gb[gb.OBJECTID_right>1].shape[0], ' points spatially match multiple segments')

2589  points spatially match no segment
124413  points spatially match 1 segment
77922  points spatially match multiple segments


In [15]:
joined.shape

(408041, 9)

In [17]:
dc_segs_statistic = dc_segments[['OBJECTID','STREETSEGID','SHAPE_Length','geometry']]
dc_segs_statistic = dc_segs_statistic.merge(joined, left_on = 'STREETSEGID', right_index=True, how='left')

In [18]:
dc_segs_statistic.head().T

,0,1,2,2,2
STREETSEGID,876,13161,8240,8240,8240
OBJECTID,1,2,3,3,3
STREETSEGID_x,876,13161,8240,8240,8240
SHAPE_Length,192.925,131.936,117.887,117.887,117.887
geometry_x,LINESTRING (-77.01239193659748 38.950836102255...,LINESTRING (-77.01163764014346 38.953383268855...,LINESTRING (-77.01157276320811 38.954558133776...,LINESTRING (-77.01157276320811 38.954558133776...,LINESTRING (-77.01157276320811 38.954558133776...
OBJECTID_left,7458,119270,76076,76076,76076
LONGITUDE,-77.0276,-77.0641,-77.0585,-77.0585,-77.0585
LATITUDE,38.9013,38.9451,38.9356,38.9356,38.9356
geometry_y,"POLYGON ((-77.02745822 38.90133936, -77.027458...","POLYGON ((-77.06395619 38.94511063, -77.063956...","POLYGON ((-77.05841170000001 38.93557396, -77....","POLYGON ((-77.05841170000001 38.93557396, -77....","POLYGON ((-77.05841170000001 38.93557396, -77...."
INITIATEDDATE,2011-10-12T16:34:55,2012-04-26T09:56:42,2012-02-09T16:14:24,2012-02-09T16:14:24,2012-02-09T16:14:24


In [20]:
dc_segs_statistic.shape

(24437, 14)

In [21]:
city_req_pts.shape

(204924, 53)

In [30]:
dc_segs_statistic.groupby('OBJECTID_left').head().T

,0,1,2,2,2,2,3,4,5,6,...,13518,13518,13518,13519,13519,13519,13520,13520,13520,13521
STREETSEGID,876,13161,8240,8240,8240,8240,10395,2216,10090,8384,...,15306,15306,15306,15284,15284,15284,15307,15307,15307,15247
OBJECTID,1,2,3,3,3,3,4,5,6,7,...,13519,13519,13519,13520,13520,13520,13521,13521,13521,13522
STREETSEGID_x,876,13161,8240,8240,8240,8240,10395,2216,10090,8384,...,15306,15306,15306,15284,15284,15284,15307,15307,15307,15247
SHAPE_Length,192.925,131.936,117.887,117.887,117.887,117.887,108.331,119.19,119.101,78.8781,...,107.921,107.921,107.921,96.5238,96.5238,96.5238,53.2962,53.2962,53.2962,40.6968
geometry_x,LINESTRING (-77.01239193659748 38.950836102255...,LINESTRING (-77.01163764014346 38.953383268855...,LINESTRING (-77.01157276320811 38.954558133776...,LINESTRING (-77.01157276320811 38.954558133776...,LINESTRING (-77.01157276320811 38.954558133776...,LINESTRING (-77.01157276320811 38.954558133776...,LINESTRING (-77.0116155077942 38.9556194835925...,LINESTRING (-77.01161670521323 38.956595348877...,LINESTRING (-77.0116172263626 38.9576690437764...,LINESTRING (-77.0116124400554 38.9587419287958...,...,LINESTRING (-77.01885842791256 38.836994168269...,LINESTRING (-77.01885842791256 38.836994168269...,LINESTRING (-77.01885842791256 38.836994168269...,LINESTRING (-76.99377942384207 38.875606235068...,LINESTRING (-76.99377942384207 38.875606235068...,LINESTRING (-76.99377942384207 38.875606235068...,LINESTRING (-77.01960436767503 38.836216481487...,LINESTRING (-77.01960436767503 38.836216481487...,LINESTRING (-77.01960436767503 38.836216481487...,LINESTRING (-77.01826374709646 38.822439566938...
OBJECTID_left,7458,119270,76076,76076,76076,76076,97451,21321,94294,79465,...,135096,135096,135096,133869,133869,133869,135098,135098,135098,133453
LONGITUDE,-77.0276,-77.0641,-77.0585,-77.0585,-77.0585,-77.0585,-77.0752,-77.0708,-77.0152,-77.0221,...,-77.0214,-77.0214,-77.0214,-77.0327,-77.0327,-77.0327,-76.9322,-76.9322,-76.9322,-77.0178
LATITUDE,38.9013,38.9451,38.9356,38.9356,38.9356,38.9356,38.959,38.9631,38.8991,38.9056,...,38.9377,38.9377,38.9377,38.9485,38.9485,38.9485,38.889,38.889,38.889,38.9043
geometry_y,"POLYGON ((-77.02745822 38.90133936, -77.027458...","POLYGON ((-77.06395619 38.94511063, -77.063956...","POLYGON ((-77.05841170000001 38.93557396, -77....","POLYGON ((-77.05841170000001 38.93557396, -77....","POLYGON ((-77.05841170000001 38.93557396, -77....","POLYGON ((-77.05841170000001 38.93557396, -77....","POLYGON ((-77.07509421 38.95903292, -77.075094...","POLYGON ((-77.07066087 38.96310065, -77.070661...","POLYGON ((-77.01512099999999 38.89907316, -77....","POLYGON ((-77.02204698999999 38.90565322, -77....",...,"POLYGON ((-77.02131177999999 38.93770468, -77....","POLYGON ((-77.02131177999999 38.93770468, -77....","POLYGON ((-77.02131177999999 38.93770468, -77....","POLYGON ((-77.03262705 38.94846986, -77.032627...","POLYGON ((-77.03262705 38.94846986, -77.032627...","POLYGON ((-77.03262705 38.94846986, -77.032627...","POLYGON ((-76.93212647999999 38.88900169, -76....","POLYGON ((-76.93212647999999 38.88900169, -76....","POLYGON ((-76.93212647999999 38.88900169, -76....","POLYGON ((-77.01769534 38.90432147, -77.017695..."
INITIATEDDATE,2011-10-12T16:34:55,2012-04-26T09:56:42,2012-02-09T16:14:24,2012-02-09T16:14:24,2012-02-09T16:14:24,2012-02-09T16:14:24,2012-03-20T20:08:29,2011-11-01T11:16:59,2012-03-15T14:23:36,2012-02-16T11:41:47,...,2012-05-22T12:29:28,2012-05-22T12:29:28,2012-05-22T12:29:28,2012-05-19T16:02:56,2012-05-19T16:02:56,2012-05-19T16:02:56,2012-05-22T12:29:30,2012-05-22T12:29:30,2012-05-22T12:29:30,2012-05-18T19:53:42


In [33]:
dc_segs_statistic = dc_segs_statistic.merge(joined, left_on = 'STREETSEGID', right_index=True, how='left')

In [34]:
dc_segs_statistic.shape

(66177, 24)

In [35]:
dc_segs_statistic.head().T

,0,1,2,2,2
STREETSEGID,876,13161,8240,8240,8240
STREETSEGID_x,876,13161,8240,8240,8240
OBJECTID,1,2,3,3,3
STREETSEGID_x,876,13161,8240,8240,8240
SHAPE_Length,192.925,131.936,117.887,117.887,117.887
geometry_x,LINESTRING (-77.01239193659748 38.950836102255...,LINESTRING (-77.01163764014346 38.953383268855...,LINESTRING (-77.01157276320811 38.954558133776...,LINESTRING (-77.01157276320811 38.954558133776...,LINESTRING (-77.01157276320811 38.954558133776...
OBJECTID_left_x,7458,119270,76076,76076,76076
LONGITUDE_x,-77.0276,-77.0641,-77.0585,-77.0585,-77.0585
LATITUDE_x,38.9013,38.9451,38.9356,38.9356,38.9356
geometry_y,"POLYGON ((-77.02745822 38.90133936, -77.027458...","POLYGON ((-77.06395619 38.94511063, -77.063956...","POLYGON ((-77.05841170000001 38.93557396, -77....","POLYGON ((-77.05841170000001 38.93557396, -77....","POLYGON ((-77.05841170000001 38.93557396, -77...."


In [37]:
dc_segs_statistic.groupby('STREETSEGID').head().T

,0,1,2,2,2,2,2,3,4,5,...,13519,13519,13519,13519,13520,13520,13520,13520,13520,13521
STREETSEGID,876,13161,8240,8240,8240,8240,8240,10395,2216,10090,...,15284,15284,15284,15284,15307,15307,15307,15307,15307,15247
STREETSEGID_x,876,13161,8240,8240,8240,8240,8240,10395,2216,10090,...,15284,15284,15284,15284,15307,15307,15307,15307,15307,15247
OBJECTID,1,2,3,3,3,3,3,4,5,6,...,13520,13520,13520,13520,13521,13521,13521,13521,13521,13522
STREETSEGID_x,876,13161,8240,8240,8240,8240,8240,10395,2216,10090,...,15284,15284,15284,15284,15307,15307,15307,15307,15307,15247
SHAPE_Length,192.925,131.936,117.887,117.887,117.887,117.887,117.887,108.331,119.19,119.101,...,96.5238,96.5238,96.5238,96.5238,53.2962,53.2962,53.2962,53.2962,53.2962,40.6968
geometry_x,LINESTRING (-77.01239193659748 38.950836102255...,LINESTRING (-77.01163764014346 38.953383268855...,LINESTRING (-77.01157276320811 38.954558133776...,LINESTRING (-77.01157276320811 38.954558133776...,LINESTRING (-77.01157276320811 38.954558133776...,LINESTRING (-77.01157276320811 38.954558133776...,LINESTRING (-77.01157276320811 38.954558133776...,LINESTRING (-77.0116155077942 38.9556194835925...,LINESTRING (-77.01161670521323 38.956595348877...,LINESTRING (-77.0116172263626 38.9576690437764...,...,LINESTRING (-76.99377942384207 38.875606235068...,LINESTRING (-76.99377942384207 38.875606235068...,LINESTRING (-76.99377942384207 38.875606235068...,LINESTRING (-76.99377942384207 38.875606235068...,LINESTRING (-77.01960436767503 38.836216481487...,LINESTRING (-77.01960436767503 38.836216481487...,LINESTRING (-77.01960436767503 38.836216481487...,LINESTRING (-77.01960436767503 38.836216481487...,LINESTRING (-77.01960436767503 38.836216481487...,LINESTRING (-77.01826374709646 38.822439566938...
OBJECTID_left_x,7458,119270,76076,76076,76076,76076,76076,97451,21321,94294,...,133869,133869,133869,133869,135098,135098,135098,135098,135098,133453
LONGITUDE_x,-77.0276,-77.0641,-77.0585,-77.0585,-77.0585,-77.0585,-77.0585,-77.0752,-77.0708,-77.0152,...,-77.0327,-77.0327,-77.0327,-77.0327,-76.9322,-76.9322,-76.9322,-76.9322,-76.9322,-77.0178
LATITUDE_x,38.9013,38.9451,38.9356,38.9356,38.9356,38.9356,38.9356,38.959,38.9631,38.8991,...,38.9485,38.9485,38.9485,38.9485,38.889,38.889,38.889,38.889,38.889,38.9043
geometry_y,"POLYGON ((-77.02745822 38.90133936, -77.027458...","POLYGON ((-77.06395619 38.94511063, -77.063956...","POLYGON ((-77.05841170000001 38.93557396, -77....","POLYGON ((-77.05841170000001 38.93557396, -77....","POLYGON ((-77.05841170000001 38.93557396, -77....","POLYGON ((-77.05841170000001 38.93557396, -77....","POLYGON ((-77.05841170000001 38.93557396, -77....","POLYGON ((-77.07509421 38.95903292, -77.075094...","POLYGON ((-77.07066087 38.96310065, -77.070661...","POLYGON ((-77.01512099999999 38.89907316, -77....",...,"POLYGON ((-77.03262705 38.94846986, -77.032627...","POLYGON ((-77.03262705 38.94846986, -77.032627...","POLYGON ((-77.03262705 38.94846986, -77.032627...","POLYGON ((-77.03262705 38.94846986, -77.032627...","POLYGON ((-76.93212647999999 38.88900169, -76....","POLYGON ((-76.93212647999999 38.88900169, -76....","POLYGON ((-76.93212647999999 38.88900169, -76....","POLYGON ((-76.93212647999999 38.88900169, -76....","POLYGON ((-76.93212647999999 38.88900169, -76....","POLYGON ((-77.01769534 38.90432147, -77.017695..."


In [38]:
dc_city_req_stats = dc_segs_statistic[['STREETSEGID', 'OBJECTID']]

In [39]:
dc_city_req_stats.shape

(66177, 2)

In [84]:
city_req_seg_counts = pd.DataFrame(dc_city_req_stats.groupby(['STREETSEGID']).size().reset_index())
city_req_seg_counts.columns = ['STREETSEGID', 'city_req_seg_count']

In [70]:
city_req_seg_counts.head().T

,0,1,2,3,4
STREETSEGID,1,2,3,4,5
city_reg_seg_count,16,16,1,1,1


In [76]:
dc_segs_statistic[dc_segs_statistic.STREETSEGID==1]

,STREETSEGID,STREETSEGID_x,OBJECTID,STREETSEGID_x,SHAPE_Length,geometry_x,OBJECTID_left_x,LONGITUDE_x,LATITUDE_x,geometry_y,...,STREETSEGID_y,OBJECTID_left_y,LONGITUDE_y,LATITUDE_y,geometry,INITIATEDDATE_y,ADDRESS_y,index_right_y,OBJECTID_right_y,STREETSEGID_y
10918,1,1,10919,1,180.26104,LINESTRING (-76.9869087694413 38.8605304682732...,2,-77.019899,38.966135,"POLYGON ((-77.01980087999999 38.96614244, -77....",...,3338.0,2,-77.019899,38.966135,"POLYGON ((-77.01980087999999 38.96614244, -77....",2014-12-13T16:13:19,SHERIDAN ST & 5TH STREET NW,3991.0,3992.0,3338.0
10918,1,1,10919,1,180.26104,LINESTRING (-76.9869087694413 38.8605304682732...,2,-77.019899,38.966135,"POLYGON ((-77.01980087999999 38.96614244, -77....",...,3338.0,2,-77.019899,38.966135,"POLYGON ((-77.01980087999999 38.96614244, -77....",2014-12-13T16:13:19,SHERIDAN ST & 5TH STREET NW,318.0,319.0,2333.0
10918,1,1,10919,1,180.26104,LINESTRING (-76.9869087694413 38.8605304682732...,2,-77.019899,38.966135,"POLYGON ((-77.01980087999999 38.96614244, -77....",...,3338.0,2,-77.019899,38.966135,"POLYGON ((-77.01980087999999 38.96614244, -77....",2014-12-13T16:13:19,SHERIDAN ST & 5TH STREET NW,319.0,320.0,143.0
10918,1,1,10919,1,180.26104,LINESTRING (-76.9869087694413 38.8605304682732...,2,-77.019899,38.966135,"POLYGON ((-77.01980087999999 38.96614244, -77....",...,3338.0,2,-77.019899,38.966135,"POLYGON ((-77.01980087999999 38.96614244, -77....",2014-12-13T16:13:19,SHERIDAN ST & 5TH STREET NW,3990.0,3991.0,6957.0
10918,1,1,10919,1,180.26104,LINESTRING (-76.9869087694413 38.8605304682732...,2,-77.019899,38.966135,"POLYGON ((-77.01980087999999 38.96614244, -77....",...,2333.0,2,-77.019899,38.966135,"POLYGON ((-77.01980087999999 38.96614244, -77....",2014-12-13T16:13:19,SHERIDAN ST & 5TH STREET NW,3991.0,3992.0,3338.0
10918,1,1,10919,1,180.26104,LINESTRING (-76.9869087694413 38.8605304682732...,2,-77.019899,38.966135,"POLYGON ((-77.01980087999999 38.96614244, -77....",...,2333.0,2,-77.019899,38.966135,"POLYGON ((-77.01980087999999 38.96614244, -77....",2014-12-13T16:13:19,SHERIDAN ST & 5TH STREET NW,318.0,319.0,2333.0
10918,1,1,10919,1,180.26104,LINESTRING (-76.9869087694413 38.8605304682732...,2,-77.019899,38.966135,"POLYGON ((-77.01980087999999 38.96614244, -77....",...,2333.0,2,-77.019899,38.966135,"POLYGON ((-77.01980087999999 38.96614244, -77....",2014-12-13T16:13:19,SHERIDAN ST & 5TH STREET NW,319.0,320.0,143.0
10918,1,1,10919,1,180.26104,LINESTRING (-76.9869087694413 38.8605304682732...,2,-77.019899,38.966135,"POLYGON ((-77.01980087999999 38.96614244, -77....",...,2333.0,2,-77.019899,38.966135,"POLYGON ((-77.01980087999999 38.96614244, -77....",2014-12-13T16:13:19,SHERIDAN ST & 5TH STREET NW,3990.0,3991.0,6957.0
10918,1,1,10919,1,180.26104,LINESTRING (-76.9869087694413 38.8605304682732...,2,-77.019899,38.966135,"POLYGON ((-77.01980087999999 38.96614244, -77....",...,143.0,2,-77.019899,38.966135,"POLYGON ((-77.01980087999999 38.96614244, -77....",2014-12-13T16:13:19,SHERIDAN ST & 5TH STREET NW,3991.0,3992.0,3338.0
10918,1,1,10919,1,180.26104,LINESTRING (-76.9869087694413 38.8605304682732...,2,-77.019899,38.966135,"POLYGON ((-77.01980087999999 38.96614244, -77....",...,143.0,2,-77.019899,38.966135,"POLYGON ((-77.01980087999999 38.96614244, -77....",2014-12-13T16:13:19,SHERIDAN ST & 5TH STREET NW,318.0,319.0,2333.0


In [71]:
city_req_seg_counts.shape

(13522, 2)

In [80]:
city_req_seg_counts

,STREETSEGID,city_reg_seg_count
0,1,16
1,2,16
2,3,1
3,4,1
4,5,1
5,6,16
6,7,1
7,8,16
8,9,1
9,10,1


In [85]:
dc_seg_city_req_statistic = dc_segments[['OBJECTID','STREETSEGID','SHAPE_Length','geometry']]
dc_seg_city_req_statistic = dc_seg_city_req_statistic.merge(city_req_seg_counts, left_on = 'STREETSEGID', right_index=True, how='left')

In [82]:
dc_seg_city_req_statistic.shape

(13522, 6)

In [86]:
dc_seg_city_req_statistic.head().T

,0,1,2,3,4
OBJECTID,1,2,3,4,5
STREETSEGID_x,876,13161,8240,10395,2216
SHAPE_Length,192.925,131.936,117.887,108.331,119.19
geometry,LINESTRING (-77.01239193659748 38.950836102255...,LINESTRING (-77.01163764014346 38.953383268855...,LINESTRING (-77.01157276320811 38.954558133776...,LINESTRING (-77.0116155077942 38.9556194835925...,LINESTRING (-77.01161670521323 38.956595348877...
STREETSEGID_y,898,14148,8458,10669,2259
city_req_seg_count,1,1,1,1,1


In [88]:
    dc_seg_city_req_statistic['city_req/len'] = dc_seg_city_req_statistic.city_req_seg_count/dc_seg_city_req_statistic.SHAPE_Length

In [91]:
dc_seg_city_req_statistic.shape

(13522, 7)

In [93]:
normalize('city_req_seg_count', dc_seg_city_req_statistic)
normalize('city_req/len', dc_seg_city_req_statistic)

In [98]:
dc_seg_city_req_statistic.head().T

,0,1,2,3,4
OBJECTID,1,2,3,4,5
STREETSEGID_x,876,13161,8240,10395,2216
SHAPE_Length,192.925,131.936,117.887,108.331,119.19
geometry,LINESTRING (-77.01239193659748 38.950836102255...,LINESTRING (-77.01163764014346 38.953383268855...,LINESTRING (-77.01157276320811 38.954558133776...,LINESTRING (-77.0116155077942 38.9556194835925...,LINESTRING (-77.01161670521323 38.956595348877...
STREETSEGID_y,898,14148,8458,10669,2259
city_req_seg_count,1,1,1,1,1
city_req/len,0.00518336,0.00757945,0.0084827,0.00923096,0.00838994
city_req_seg_count_norm,0.157293,0.157293,0.157293,0.157293,0.157293
city_req/len_norm,0.00411724,0.00601333,0.00672692,0.00731758,0.00665367


In [114]:
dc_seg_city_req_statistic_drop = dc_seg_city_req_statistic.dropna()

In [115]:
def gradient_color(percent):
    min_color = np.array([255,255,255])
    max_color = np.array([178,34,34])
    return '#%02x%02x%02x' % tuple([int(k) for k in min_color+(max_color-min_color)*percent])

def write_var(col_name, var_name, df,f):
    df['color']=df[col_name].apply(gradient_color)
    f.write('var %s = %s;\n' % (var_name, df.to_json()))

with open('../visualization/Volumes of City Requests.js','w+') as f:
    write_var('city_req_seg_count_norm', 'city_req_count', dc_seg_city_req_statistic_drop,f)
    write_var('city_req/len_norm', 'city_req_count_norm', dc_seg_city_req_statistic_drop,f)

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [116]:
dc_seg_city_req_statistic_drop[dc_seg_city_req_statistic_drop.STREETSEGID_x==6316]

,OBJECTID,STREETSEGID_x,SHAPE_Length,geometry,STREETSEGID_y,city_req_seg_count,city_req/len,city_req_seg_count_norm,city_req/len_norm,color
12898,12899,6316,661.947921,LINESTRING (-77.03665083891831 38.886935862966...,6477.0,1.0,0.001511,0.157293,0.001202,#fefefe


In [117]:
dc_seg_city_req_statistic_drop[dc_seg_city_req_statistic_drop.STREETSEGID_y==6316]

,OBJECTID,STREETSEGID_x,SHAPE_Length,geometry,STREETSEGID_y,city_req_seg_count,city_req/len,city_req_seg_count_norm,city_req/len_norm,color
8696,8697,6160,97.850095,LINESTRING (-77.00204275692717 38.879293174632...,6316.0,16.0,0.163515,0.64293,0.120608,#f5e4e4
